In [ ]:
import pathlib

import numpy as nps
import xarray as xr
import seaborn as sns
from matplotlib import pyplot as plt

from weight_estimation import get_edge_probs
from weight_estimation.util import get_defects, get_syndromes

Define a few constants

In [ ]:
BASIS = "Z"
LOG_STATE = 0
NUM_SHOTS = 100000
NUM_ROUNDS = 20

In [ ]:
NOTEBOOK_DIR: str = pathlib.Path.cwd()

# Load the simulated data

In [ ]:
exp_name = f"surf-code_d3_b{BASIS}_s{LOG_STATE}_n{NUM_SHOTS}_r{NUM_ROUNDS}"
exp_folder = NOTEBOOK_DIR / "data" / exp_name

measurements = xr.open_dataset(exp_folder / "measurements.nc")

syndromes = get_syndromes(measurements.anc_meas)
defects = get_defects(syndromes)

# Edge probability estimation

In [ ]:
z_anc = ["Z1", "Z2", "Z3", "Z4"]
sel_defects = defects.sel(anc_qubit=z_anc)
defects_vec = sel_defects.stack(node=("anc_qubit", "qec_round"))

edge_probs = get_edge_probs(sel_defects, normalise=False)


In [ ]:
num_anc = sel_defects.anc_qubit.size
num_rounds = defects.qec_round.size
num_nodes = num_anc * num_rounds

probs = edge_probs.edge.values
fig, ax = plt.subplots(dpi=200)

with sns.axes_style("white"):
    full_scale_img = sns.heatmap(
        probs,
        ax=ax,
        vmin=0,
        vmax=0.01,
        linewidths=0.01,
        linecolor="gray",
        square=True,
        cmap="Blues",
    )
full_scale_img.axes.tick_params(axis="both", which="both", direction="out")

def label_anc(anc_q):
    return f"{anc_q[0]}$_{{{anc_q[1:]}}}$"

major_labels = list(map(label_anc, z_anc))
minor_labels = defects_vec.qec_round.values

minor_ticks = list(range(num_nodes))
major_ticks = list(range(0, num_nodes, num_rounds))

ax.set_xticks(major_ticks, labels=major_labels)
ax.set_xticks(minor_ticks, minor=True)

ax.set_yticks(major_ticks, labels=major_labels)
ax.set_yticks(minor_ticks, minor=True)

ax.invert_yaxis()
plt.show()